In [1]:
# Code from https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/

import numpy as np
import pandas as pd
from nltk.util import ngrams
import collections
import itertools
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding = 'utf8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [3]:
in_filename = 'Data/GlobalVoices_en.txt'
raw_text = load_doc(in_filename)
raw_text = raw_text.lower()
lines = raw_text.split('\n')

In [4]:
# integer encode sequences of characters
length = 10
sequences = list()
k = 0
j = 0
for line in lines: 
    if j < 25000:
        k += 1
        if line[-1:] == '.' or line[-1:] == '?' or line[-1:] == '!' or line[-1:] == '"':
            j += 1
            for i in range(length, len(line)):
                seq = raw_text[i-length:i+1]
                sequences.append(seq)

In [89]:
#out_filename = 'char_sequences.txt'
#save_doc(sequences, out_filename)

In [5]:
print(k)
print(j)
print(len(sequences))

33485
25000
2847853


In [6]:
print('Length of Original Corpus: %s' % len(lines))
print('Length of Tokenized Sequences in LM Corpus: %s' % len(sequences))
print('Sentence line we stop at: %s' % k)
print('Number of Sentences in our dataset: %s' % j)

Length of Original Corpus: 979735
Length of Tokenized Sequences in LM Corpus: 2847853
Sentence line we stop at: 33485
Number of Sentences in our dataset: 25000


In [7]:
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [8]:
encoded_sequences = list()
for row in sequences:
    # integer encode line
    encoded_seq = [mapping[char] for char in row]
    # store
    encoded_sequences.append(encoded_seq)

In [9]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 3338


In [10]:
encoded_sequences[0:100]

[[46, 44, 54, 57, 46, 48, 40, 1, 13, 1, 57],
 [44, 54, 57, 46, 48, 40, 1, 13, 1, 57, 60],
 [54, 57, 46, 48, 40, 1, 13, 1, 57, 60, 58],
 [57, 46, 48, 40, 1, 13, 1, 57, 60, 58, 58],
 [46, 48, 40, 1, 13, 1, 57, 60, 58, 58, 48],
 [48, 40, 1, 13, 1, 57, 60, 58, 58, 48, 40],
 [40, 1, 13, 1, 57, 60, 58, 58, 48, 40, 1],
 [1, 13, 1, 57, 60, 58, 58, 48, 40, 1, 27],
 [13, 1, 57, 60, 58, 58, 48, 40, 1, 27, 1],
 [1, 57, 60, 58, 58, 48, 40, 1, 27, 1, 59],
 [57, 60, 58, 58, 48, 40, 1, 27, 1, 59, 47],
 [60, 58, 58, 48, 40, 1, 27, 1, 59, 47, 44],
 [58, 58, 48, 40, 1, 27, 1, 59, 47, 44, 1],
 [58, 48, 40, 1, 27, 1, 59, 47, 44, 1, 53],
 [48, 40, 1, 27, 1, 59, 47, 44, 1, 53, 40],
 [40, 1, 27, 1, 59, 47, 44, 1, 53, 40, 52],
 [1, 27, 1, 59, 47, 44, 1, 53, 40, 52, 44],
 [27, 1, 59, 47, 44, 1, 53, 40, 52, 44, 58],
 [1, 59, 47, 44, 1, 53, 40, 52, 44, 58, 1],
 [59, 47, 44, 1, 53, 40, 52, 44, 58, 1, 54],
 [47, 44, 1, 53, 40, 52, 44, 58, 1, 54, 45],
 [44, 1, 53, 40, 52, 44, 58, 1, 54, 45, 1],
 [1, 53, 40, 52, 44, 

In [12]:
# separate into input and output
encoded_sequences = np.array(encoded_sequences) #, shape = (len(sequences),length))
X, y = encoded_sequences[:,:-1], encoded_sequences[:,-1]
new_sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(new_sequences)
y = to_categorical(y, num_classes=vocab_size)

In [15]:
X.shape

(2847853, 10)

In [14]:
y

array([57, 60, 58, ..., 48, 59, 57])

In [18]:
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

TypeError: 'int' object is not iterable